<a href="https://colab.research.google.com/github/patchu-madathil/FastAI/blob/master/catsanddogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fast AI Lesson 1 


In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import datasets
from fastai.vision import *
from fastai.metrics import error_rate

path = untar_data(URLs.PETS)




In [5]:
path.absolute()
path.ls()




[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [0]:
annotation_path = path/'annotations'
image_path = path/'images'

In [8]:
fnames = get_image_files(image_path)
fnames[:5]

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/newfoundland_196.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Persian_36.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bengal_103.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bombay_3.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_119.jpg')]

In [0]:
?doc